In [ ]:
# ===============================================
# DROP-IN REPLACEMENT for the cell that ADDS/plots
# the sinogram WITH ring artifacts (ROW-based).
# Target notebook: quantum supremacy(Body CT image 50x50 - ring artifacts).ipynb
# Assumes: fn2, theta, radon are already defined.
# ===============================================

import yaml, numpy as np
import matplotlib.pyplot as plt

# 1) Load config and pick a case
with open("configs/config.yaml") as f:   
    CFG = yaml.safe_load(f)

CASE = "50x50_ra_10"  # choose a case name from CFG["cases"]
case = next(c for c in CFG["cases"] if c["name"] == CASE)

ring_rows = case.get("ring_artifact", {}).get("ring_rows")
ring_vals = case.get("ring_artifact", {}).get("artifact_strengths")

# 2) Deterministic ring injection from config (no randomness)
def add_ring_artifact_from_config(sinogram, ring_rows, strengths):
    """
    Multiply specified sinogram ROWS by given strengths.
    Returns:
      artifact_sinogram, artifacts_map, used_rows
    """
    assert ring_rows is not None and strengths is not None, "config requires ring_rows/artifact_strengths"
    rr = np.asarray(ring_rows, dtype=int)
    vv = np.asarray(strengths, dtype=float)
    assert rr.size == vv.size, "row indices and strengths length mismatch"

    S = sinogram.copy()
    A = np.zeros_like(S)
    S[rr, :] *= vv[:, None]
    A[rr, :]  =  vv[:, None]
    return S, A, rr

# 3) Build sinogram, inject, and PLOT (replaces original plotting cell)
sino = radon(fn2, theta)
artifact_sinogram, artifacts, wh_artifacts = add_ring_artifact_from_config(sino, ring_rows, ring_vals)

zero_rows = np.array(sorted(wh_artifacts))
print("Rows to be set to zero:", zero_rows)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.title("Original Sinogram"); plt.imshow(sino, cmap='gray', aspect='auto'); plt.colorbar()
plt.subplot(1,2,2); plt.title("Sinogram with Ring Artifacts"); plt.imshow(artifact_sinogram, cmap='gray', aspect='auto'); plt.colorbar()
plt.tight_layout(); plt.show()


In [ ]:
# ===============================================
# DROP-IN REPLACEMENT for the cell that ADDS/plots
# the sinogram WITH missing projection angles (COLUMN-based).
# Target notebook: quantum supremacy(Body CT image 50x50 - limited_angles).ipynb
# Assumes: fn2, theta, radon are already defined.
# ===============================================

import yaml, numpy as np
import matplotlib.pyplot as plt

# 1) Load config and pick a case with "missing_projection_columns"
with open("config.yaml") as f:   # or "configs/config.yaml"
    CFG = yaml.safe_load(f)

CASE = "50x50_missing_angles_set1"  # choose a case name from CFG["cases"]
case = next(c for c in CFG["cases"] if c["name"] == CASE)

missing_cols = case.get("missing_projection_columns")

# 2) Zero specified columns to emulate missing angles
def apply_missing_angles_from_config(sinogram, cols):
    """
    Zero-out specified sinogram COLUMNS.
    """
    if not cols:
        return sinogram
    S = sinogram.copy()
    S[:, cols] = 0
    return S

# 3) Build sinogram, apply missing angles, and PLOT (replaces original plotting cell)
sino = radon(fn2, theta)
artifact_sinogram = apply_missing_angles_from_config(sino, missing_cols)

zero_cols = missing_cols
print("Columns to be set to zero:", zero_cols)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.title("Original Sinogram"); plt.imshow(sino, cmap='gray', aspect='auto'); plt.colorbar()
plt.subplot(1,2,2); plt.title("Sinogram with Missing Angles"); plt.imshow(artifact_sinogram, cmap='gray', aspect='auto'); plt.colorbar()
plt.tight_layout(); plt.show()
